**Total Charges Per Billing Rate**


In [0]:
create table if not exists billings 
(
emp_name varchar(10),
bill_date date,
bill_rate int
);
insert into billings values
('Sachin','1990-01-01',25)
,('Sehwag' ,'1989-01-01', 15)
,('Dhoni' ,'1989-01-01', 20)
,('Sachin' ,'1991-02-05', 30)
;

create table if not exists HoursWorked 
(
emp_name varchar(20),
work_date date,
bill_hrs int
);
insert into HoursWorked values
('Sachin', '1990-07-01' ,3)
,('Sachin', '1990-08-01', 5)
,('Sehwag','1990-07-01', 2)
,('Sachin','1991-07-01', 4);

num_affected_rows,num_inserted_rows
4,4


In [0]:
select * from billings;

emp_name,bill_date,bill_rate
Sachin,1990-01-01,25
Sehwag,1989-01-01,15
Dhoni,1989-01-01,20
Sachin,1991-02-05,30


In [0]:
select * from HoursWorked;

emp_name,work_date,bill_hrs
Sachin,1990-07-01,3
Sachin,1990-08-01,5
Sehwag,1990-07-01,2
Sachin,1991-07-01,4


In [0]:
select * from billings

emp_name,bill_date,bill_rate
Sachin,1990-01-01,25
Sehwag,1989-01-01,15
Dhoni,1989-01-01,20
Sachin,1991-02-05,30


In [0]:
select 
*,
lead(bill_date,1) over(partition by emp_name order by bill_date) as lag_col
from
billings

emp_name,bill_date,bill_rate,lag_col
Dhoni,1989-01-01,20,null
Sachin,1990-01-01,25,1991-02-05
Sachin,1991-02-05,30,null
Sehwag,1989-01-01,15,null


In [0]:
select
*,
lead(bill_date,1,'9999-12-31') over (partition by emp_name order by bill_date) as lag_col
from
billings

emp_name,bill_date,bill_rate,lag_col
Dhoni,1989-01-01,20,9999-12-31
Sachin,1990-01-01,25,1991-02-05
Sachin,1991-02-05,30,9999-12-31
Sehwag,1989-01-01,15,9999-12-31


In [0]:
select
*,
substr(lead(date_add(day,-1,bill_date),1,'9999-12-31') over (partition by emp_name order by bill_date),1,10) as lag_col
from
billings

emp_name,bill_date,bill_rate,lag_col
Dhoni,1989-01-01,20,9999-12-31
Sachin,1990-01-01,25,1991-02-04
Sachin,1991-02-05,30,9999-12-31
Sehwag,1989-01-01,15,9999-12-31


In [0]:
with date_range as
(select
*,
substr(lead(date_add(day,-1,bill_date),1,'9999-12-31') over (partition by emp_name order by bill_date),1,10) as lag_col
from
billings
)


select
dr.emp_name,
sum(dr.bill_rate*hw.bill_hrs) as total_charges
from date_range dr   
join hoursworked hw
on dr.emp_name = hw.emp_name and hw.work_date between dr.bill_date and dr.lag_col
group by dr.emp_name

emp_name,total_charges
Sachin,320
Sehwag,30
